# **Quantium Virtual Intership - Data Analytics**

---



## Project Background

Category Manager for Chips(A Chips Making Company) wants to better understand the types of customers who purchase Chips and their purchasing behaviour within the region.

The insights from this analysis will feed into the supermarket’s strategic plan for the chip category in the next half year.

---



### Task Breakdown
1. Examine & Explore the data
2. Clean up the data 
3. Analyse customer segments – Define the metrics. Look at total sales, drivers of sales, where the highest sales are coming from etc. Explore the data, create charts and graphs as well as noting any interesting trends and/or insights.
4.Deep dive into customer segments – define recommendation from the insights, determine which segments client should be targeting, if packet sizes are relative and form an overall conclusion based on your analysis. 


In [66]:
### Importing Necesary Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
import os

%matplotlib inline

#### The transaction data is a bit large so to simplify loading it into the notebook, it's more effiecient to programatically load it from github, using the request library.

In [67]:
url_data = 'https://github.com/idowujames/Quantium-Virtual-Intership/blob/main/QVI_transaction_data.xlsx?raw=true'

r_data = requests.get(url_data)

with open ('QVI_transaction_data.xlsx', 'wb') as file:
  file.write(r_data.content)

In [68]:
### Loading the dataframe 

df_trans = pd.read_excel('QVI_transaction_data.xlsx')

df_trans.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


### Exploring the QVI transaction data

In [69]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


### Structure of the dataset

> This dataset consist of **264,836** entries with **8** columns

> There are no missing values in the observations but some of the columns are not in optimised datatypes for analysis.

In [70]:
df_trans.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,264836.000000,264836.00000,2.648360e+05,2.648360e+05,264836.000000,264836.000000,264836.000000
mean,43464.036260,135.08011,1.355495e+05,1.351583e+05,56.583157,1.907309,7.304200
std,105.389282,76.78418,8.057998e+04,7.813303e+04,32.826638,0.643654,3.083226
min,43282.000000,1.00000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.500000
25%,43373.000000,70.00000,7.002100e+04,6.760150e+04,28.000000,2.000000,5.400000
50%,43464.000000,130.00000,1.303575e+05,1.351375e+05,56.000000,2.000000,7.400000
75%,43555.000000,203.00000,2.030942e+05,2.027012e+05,85.000000,2.000000,9.200000
max,43646.000000,272.00000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000


## Getting the number of stores, customers and products in the data to get a better understanding of the dataset

In [75]:
# Checking for number of unique stores in the data
df_trans['STORE_NBR'].nunique()

272

In [76]:
# Checking for number of unique customers in the data
df_trans['LYLTY_CARD_NBR'].nunique()

72637

In [84]:
# Checking for number of unique products in the data
print(df_trans['PROD_NAME'].nunique())

print('--------------')

# Checking if the unique number of product_NAME matches the number of product_NBR
df_trans['PROD_NAME'].nunique() == df_trans['PROD_NBR'].nunique()

114
--------------


True

In [77]:
df_trans['TXN_ID'].nunique()

263127

> There are **272 stores**, **114 Products** and **72,637 Customers** in this dataset

In [71]:
df_trans.sample(16)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
165980,43642,200,200155,199512,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
134697,43553,264,264461,263261,29,French Fries Potato Chips 175g,1,3.0
108761,43357,129,129077,132663,81,Pringles Original Crisps 134g,2,7.4
196810,43405,164,164182,164907,77,Doritos Corn Chips Nacho Cheese 170g,2,8.8
39143,43526,80,80155,78879,44,Thins Chips Light& Tangy 175g,2,6.6
19050,43582,63,63193,60649,46,Kettle Original 175g,2,10.8
207847,43407,81,81253,80855,109,Pringles Barbeque 134g,2,7.4
261136,43476,106,106082,107242,106,Natural ChipCo Hony Soy Chckn175g,2,6.0
234090,43444,160,160250,161726,44,Thins Chips Light& Tangy 175g,2,6.6
43802,43298,125,125199,129089,99,Pringles Sthrn FriedChicken 134g,2,7.4


### Checking for error entries or outliers in the dataset

In [85]:
# Checking if there are errors in the products thats were captures
# We are only interested in chips and not any other products

df_trans['PROD_NAME'].unique()

array(['Natural Chip        Compny SeaSalt175g',
       'CCs Nacho Cheese    175g',
       'Smiths Crinkle Cut  Chips Chicken 170g',
       'Smiths Chip Thinly  S/Cream&Onion 175g',
       'Kettle Tortilla ChpsHny&Jlpno Chili 150g',
       'Old El Paso Salsa   Dip Tomato Mild 300g',
       'Smiths Crinkle Chips Salt & Vinegar 330g',
       'Grain Waves         Sweet Chilli 210g',
       'Doritos Corn Chip Mexican Jalapeno 150g',
       'Grain Waves Sour    Cream&Chives 210G',
       'Kettle Sensations   Siracha Lime 150g',
       'Twisties Cheese     270g', 'WW Crinkle Cut      Chicken 175g',
       'Thins Chips Light&  Tangy 175g', 'CCs Original 175g',
       'Burger Rings 220g', 'NCC Sour Cream &    Garden Chives 175g',
       'Doritos Corn Chip Southern Chicken 150g',
       'Cheezels Cheese Box 125g', 'Smiths Crinkle      Original 330g',
       'Infzns Crn Crnchers Tangy Gcamole 110g',
       'Kettle Sea Salt     And Vinegar 175g',
       'Smiths Chip Thinly  Cut Original 175g', 'K

### *It appears not all the products in the dataset are chips. Some appear to be salsa sour*

In [98]:
# Checking out entries in the dataset that are salsa

salsa = df_trans[df_trans['PROD_NAME'].str.lower().str.contains('salsa')]

salsa

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
5,43604,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
25,43600,39,39144,35506,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
32,43605,45,45127,41122,64,Red Rock Deli SR Salsa & Mzzrlla 150g,2,5.4
44,43330,56,56013,50090,39,Smiths Crinkle Cut Tomato Salsa 150g,1,2.6
63,43600,82,82480,82047,101,Doritos Salsa Medium 300g,1,2.6
...,...,...,...,...,...,...,...,...
264675,43575,265,265103,263419,59,Old El Paso Salsa Dip Tomato Med 300g,1,5.1
264678,43554,265,265111,263428,35,Woolworths Mild Salsa 300g,1,1.5
264719,43401,266,266278,264104,39,Smiths Crinkle Cut Tomato Salsa 150g,1,2.6
264734,43476,267,267324,264374,41,Doritos Salsa Mild 300g,1,2.6


### Filter out all entries that are salsa from the dataset

In [99]:
df_trans[~df_trans.isin(salsa).all(1)]

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...
264831,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [ ]:
df_pur = pd.read_csv('https://raw.githubusercontent.com/idowujames/Quantium-Virtual-Intership/main/QVI_purchase_behaviour.csv')

df_pur.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
